In [9]:
import pandas as pd
import os
import json
from functools import singledispatch

In [10]:
date = "2022-01-22"

In [11]:
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
  'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [12]:
data_min_json={}
for k,v in STATE_NAMES.items():
    data_min_json[k]={"districts":{},"delta":{},"delta7":{},"delta21_14":{},"meta":{},"total":{}}

In [13]:
def number_generation(df,col_name,col_value,field):
    value=df.loc[df[col_name]==col_value,field]
    value.reset_index(inplace=True,drop=True)
    if value.isna().all():
        value = "null"
    try:
        value = int(value[0])
    except:
        # raise
        value = value[0]
    return value

In [14]:
import json
import traceback
from datetime import datetime
import time

def addLogging(logDict:dict):
    loggingsFile = '../log.json'

    with open(loggingsFile) as f:
        data = json.load(f)
        #data = []

    data.append(logDict)

    with open(loggingsFile, 'w') as f:
        json.dump(data, f)

def currentTimeUTC(date):
    # return time.mktime(datetime.strptime(datetime.now().strftime('%d/%m/%Y'),'%d/%m/%Y').timetuple())
    return int(time.mktime(datetime.strptime(date,'%Y-%m-%d').timetuple()))
    #return datetime.now().strftime('%d/%m/%Y %H:%M:%S')

def updateJSONLog(stateName,ttValue,stateValue,date):
    addLogging({
      "update": stateName + ":\n State totals for confirmed cases reported on https://www.mygov.in/covid-19 ("+str(ttValue)+") different from district total for confirmed cases reported in state government bulletin ("+str(stateValue)+")",
      "timestamp": currentTimeUTC(date)
   })

    # State totals for confirmed cases reported on https://www.mygov.in/covid-19 (xxx) different from district total for confirmed cases reported in state government bulletin (yyy)
# try:
#     print(5/0)
# except ZeroDivisionError:
#     fullTraceback = str(traceback.format_exc())
#     addLogging({'timestamp': currentTimeUTC(), 'level': 'error', 'traceback': fullTraceback})

In [15]:
for k,v in data_min_json.items():
    print(k)
    df = pd.read_csv("../RAWCSV/"+date+"/"+k+"_final.csv")
    df_tt = pd.read_csv("../RAWCSV/"+date+"/TT_final.csv")
    # df = pd.read_csv('datamin.csv',header=[1])
    for district in df["District"]:
        try:
            district_dict={district:{"delta":{"confirmed":number_generation(df,"District",district,"deltaConfirmedForDistrict"),
                                              "recovered":number_generation(df,"District",district,"deltaRecoveredForDistrict"),
                                              "deceased":number_generation(df,"District",district,"deltaDeceasedForDistrict"),
                                              "tested":number_generation(df,"District",district,"deltaTestedForDistrict"),
                                              "vaccinated1":number_generation(df,"District",district,"deltaVaccinated1ForDistrict"),
                                              "vaccinated2":number_generation(df,"District",district,"deltaVaccinated2ForDistrict"),
                                              "vaccinated3":number_generation(df,"District",district,"deltaVaccinated3ForDistrict")
                                             },
                                    # },#}
            # district_dict={district:{
                                     "delta7":{
                                    "confirmed": number_generation(df,"District",district,"7DmaConfirmedForDistrict"),
                                    "deceased": number_generation(df,"District",district,"7DmaDeceasedForDistrict"),
                                    "recovered": number_generation(df,"District",district,"7DmaRecoveredForDistrict"),
                                    "tested": number_generation(df,"District",district,"7DmaTestedForDistrict"),
                                    "Other": number_generation(df,"District",district,"7DmaOtherForDistrict"),
                                    "vaccinated1": number_generation(df,"District",district,"7DmaVaccinated1ForDistrict"),
                                    "vaccinated2": number_generation(df,"District",district,"7DmaVaccinated2ForDistrict"),
                                    "vaccinated3": number_generation(df,"District",district,"7DmaVaccinated3ForDistrict")
                                   },
            #                }
            #               }
                           "meta":{"population": number_generation(df,"District",district,"districtPopulation"),
                                 "tested": {
                                  "last_updated": number_generation(df,"State/UTCode",k,'last_updated'),
                                  "source": number_generation(df,"State/UTCode",k,'tested_source_state'),
                                 },
                                 "notes": number_generation(df,"State/UTCode",k,'notesForDistrict')
                           },
                            "total":{
                               "confirmed": number_generation(df,"District",district,'cumulativeConfirmedNumberForDistrict'),
                                 "deceased": number_generation(df,"District",district,'cumulativeDeceasedNumberForDistrict'),
                                 "recovered": number_generation(df,"District",district,'cumulativeRecoveredNumberForDistrict'),
                                "other": number_generation(df,"District",district,'cumulativeOtherNumberForDistrict'),
                                 "tested": number_generation(df,"District",district,'cumulativeTestedNumberForDistrict'),
                                 "vaccinated1": number_generation(df,"District",district,'cumulativeVaccinated1NumberForDistrict'),
                                "vaccinated2": number_generation(df,"District",district,'cumulativeVaccinated2NumberForDistrict'),
                                "vaccinated3": number_generation(df,"District",district,'cumulativeVaccinated3NumberForDistrict'),
                           }}
                          }
            
            
        except KeyError as e:
            print(district,e)
            pass
        else:
            data_min_json[k]["districts"].update(district_dict)
            
            
    print("State:"+str(number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState')))
    print("TT   :"+str(number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict')))
    if k != "TT":
        if number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState') != number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict'):
            updateJSONLog(STATE_NAMES[k],number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict'),number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState'),date)
            # pass
    if k != "TT":
        data_min_json[k]["delta"]["confirmed"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaConfirmedForDistrict')
        data_min_json[k]["delta"]["deceased"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaDeceasedForDistrict')
        data_min_json[k]["delta"]["recovered"]=number_generation(df_tt,"District",STATE_NAMES[k],'deltaRecoveredForDistrict')
    else:
        data_min_json[k]["delta"]["confirmed"]=number_generation(df,"State/UTCode",k,'deltaConfirmedForState')
        data_min_json[k]["delta"]["deceased"]=number_generation(df,"State/UTCode",k,'deltaDeceasedForState')
        data_min_json[k]["delta"]["recovered"]=number_generation(df,"State/UTCode",k,'deltaRecoveredForState')
    
    data_min_json[k]["delta"]["vaccinated1"]=number_generation(df,"State/UTCode",k,'deltaVaccinated1ForState')
    data_min_json[k]["delta"]["vaccinated2"]=number_generation(df,"State/UTCode",k,'deltaVaccinated2ForState')
    data_min_json[k]["delta"]["vaccinated3"]=number_generation(df,"State/UTCode",k,'deltaVaccinated3ForState')
    data_min_json[k]["delta"]["tested"]=number_generation(df,"State/UTCode",k,'deltaTestedForState')
    # if k != "TT":
    #     data_min_json[k]["delta"]["tested"]=number_generation(df,"State/UTCode",k,'deltaTestedForState')
    # else:
    #     data_min_json[k]["delta"]["tested"]=0
    
    data_min_json[k]["delta7"]["confirmed"]=number_generation(df,"State/UTCode",k,'7DmaConfirmedForState')
    data_min_json[k]["delta7"]["deceased"]=number_generation(df,"State/UTCode",k,'7DmaDeceasedForState')
    data_min_json[k]["delta7"]["recovered"]=number_generation(df,"State/UTCode",k,'7DmaRecoveredForState')
    data_min_json[k]["delta7"]["vaccinated1"]=number_generation(df,"State/UTCode",k,'7DmaVaccinated1ForState')
    data_min_json[k]["delta7"]["vaccinated2"]=number_generation(df,"State/UTCode",k,'7DmaVaccinated2ForState')
    data_min_json[k]["delta7"]["vaccinated3"]=number_generation(df,"State/UTCode",k,'7DmaVaccinated3ForState')
    data_min_json[k]["delta7"]["tested"]=number_generation(df,"State/UTCode",k,'7DmaTestedForState')
    data_min_json[k]["delta7"]["other"]=number_generation(df,"State/UTCode",k,'7DmaOtherForState')
    data_min_json[k]["delta21_14"]={"confirmed":number_generation(df,"State/UTCode",k,'delta21_14confirmedForState')}
    
    data_min_json[k]["meta"]={"date":number_generation(df,"State/UTCode",k,'Date'),
                             "last_updated":number_generation(df,"State/UTCode",k,'last_updated'),
                              "notes":number_generation(df,"State/UTCode",k,'notesForState'),
                              "population":number_generation(df,"State/UTCode",k,'statePopulation'),
                              "tested":{"date":number_generation(df,"State/UTCode",k,'last_updated'),
                                        "source":number_generation(df,"State/UTCode",k,'tested_source_state')
                              }
                             }
    if k != "TT":
        data_min_json[k]["total"]["confirmed"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeConfirmedNumberForDistrict')
        data_min_json[k]["total"]["deceased"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeDeceasedNumberForDistrict')
        data_min_json[k]["total"]["recovered"] = number_generation(df_tt,"District",STATE_NAMES[k],'cumulativeRecoveredNumberForDistrict')
        data_min_json[k]["total"]["other"] = number_generation(df,"District",district,'cumulativeOtherNumberForState')
    else:
        data_min_json[k]["total"]["confirmed"] = number_generation(df,"State/UTCode",k,'cumulativeConfirmedNumberForState')
        data_min_json[k]["total"]["deceased"] = number_generation(df,"State/UTCode",k,'cumulativeDeceasedNumberForState')
        data_min_json[k]["total"]["recovered"] = number_generation(df,"State/UTCode",k,'cumulativeRecoveredNumberForState')
        
    
    data_min_json[k]["total"]["tested"] = number_generation(df,"State/UTCode",k,'cumulativeTestedNumberForState')
    data_min_json[k]["total"]["vaccinated1"] = number_generation(df,"State/UTCode",k,'cumulativeVaccinated1NumberForState')
    data_min_json[k]["total"]["vaccinated2"] = number_generation(df,"State/UTCode",k,'cumulativeVaccinated2NumberForState')
    data_min_json[k]["total"]["vaccinated3"] = number_generation(df,"State/UTCode",k,'cumulativeVaccinated3NumberForState')

AP
State:2166194
TT   :2166194
AR
State:59162
TT   :59162
AS
State:690534
TT   :690534
BR
State:808754
TT   :808754
CT
State:1091868
TT   :1091868
GA
State:227856
TT   :227856
GJ
State:1045938
TT   :1045938
HR
State:901303
TT   :901303
HP
State:259566
TT   :259566
JH
State:419158
TT   :419158
KA
State:3467472
TT   :3467472
KL
State:5597648
TT   :5597648
MP
State:882906
TT   :882906
MH
State:7466420
TT   :7466420
MN
State:129661
TT   :129661
ML
State:88141
TT   :88141
MZ
State:159653
TT   :159653
NL
State:33478
TT   :33478
OR
State:1196140
TT   :1196140
PB
State:707847
TT   :707847
RJ
State:1115790
TT   :1115790
SK
State:36765
TT   :36765
TN
State:3103410
TT   :3103410
TG
State:731212
TT   :731212
TR
State:97930
TT   :97933
UT
State:396674
TT   :396674
UP
State:1933165
TT   :1933165
WB
State:1958265
TT   :1958265
AN
State:9277
TT   :9277
CH
State:84884
TT   :84884
DN
State:11172
TT   :11172
DL
State:1782514
TT   :1782514
JK
State:390949
TT   :390949
LA
State:24496
TT   :24496
LD
State:1

In [16]:
@singledispatch
def remove_null_bool(ob):
    return ob

@remove_null_bool.register(list)
def _process_list(ob):
    return [remove_null_bool(v) for v in ob]

@remove_null_bool.register(dict)
def _process_list(ob):
    return {k: remove_null_bool(v) for k, v in ob.items()
            if v is not None and v is not 0 and v is not 'n' and v is not {}}

with open('data.min_swi.json', 'w') as json_file:
    json.dump(remove_null_bool(data_min_json), json_file)

# Stop here

In [42]:
import requests, json

In [43]:
url = requests.get("https://data.covid19india.org/v4/min/data.min.json")
text = url.text

In [44]:
C19I_data = json.loads(text)

In [35]:
compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))

False

In [36]:
print('Compare JSON result is: {0}'.format(
		compare_json_data(C19I_data['RJ'],remove_null_bool(data_min_json))
	))

Compare JSON result is: False


In [21]:
!pip install jsondiff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for jsondiff, since package 'wheel' is not installed.
    Running setup.py install for jsondiff: started
    Running setup.py install for jsondiff: finished with status 'done'
